In [4]:
import xarray as xr
import os

# Path to the directory containing the NetCDF files
data_dir = '/teamspace/studios/this_studio/extracted_files/'

# Search for all .nc files in the directory
nc_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.nc')]


In [5]:
nc_files

['/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19910901000000-TCDR-v202212.0.0.nc',
 '/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19911001000000-TCDR-v202212.0.0.nc',
 '/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19911101000000-TCDR-v202212.0.0.nc',
 '/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19911201000000-TCDR-v202212.0.0.nc',
 '/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19920101000000-TCDR-v202212.0.0.nc',
 '/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19920201000000-TCDR-v202212.0.0.nc',
 '/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19920301000000-TCDR-v202212.0.0.nc',
 '/teamspace/studios/this_studio/extracted_files/C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19920401000000-TCDR-v

In [6]:
import xarray as xr 

# Use xarray to open multiple NetCDF files
ds = xr.open_mfdataset(nc_files, combine='by_coords')


In [7]:
ds

<xarray.Dataset> Size: 8GB
Dimensions:     (time: 376, lat: 720, lon: 1440)
Coordinates:
  * lat         (lat) float32 3kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon         (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time        (time) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
Data variables:
    sm          (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sensor      (time, lat, lon) float64 3GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    freqbandID  (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    nobs        (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [8]:
ds = ds.load()

In [9]:
ds

<xarray.Dataset> Size: 8GB
Dimensions:     (time: 376, lat: 720, lon: 1440)
Coordinates:
  * lat         (lat) float32 3kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon         (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time        (time) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
Data variables:
    sm          (time, lat, lon) float32 2GB nan nan nan nan ... nan nan nan nan
    sensor      (time, lat, lon) float64 3GB nan nan nan nan ... nan nan nan nan
    freqbandID  (time, lat, lon) float32 2GB nan nan nan nan ... nan nan nan nan
    nobs        (time, lat, lon) float32 2GB nan nan nan nan ... nan nan nan nan
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [10]:
# ds = ds.sel(lon=slice(25, 60))

In [11]:
ds

<xarray.Dataset> Size: 8GB
Dimensions:     (time: 376, lat: 720, lon: 1440)
Coordinates:
  * lat         (lat) float32 3kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon         (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time        (time) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
Data variables:
    sm          (time, lat, lon) float32 2GB nan nan nan nan ... nan nan nan nan
    sensor      (time, lat, lon) float64 3GB nan nan nan nan ... nan nan nan nan
    freqbandID  (time, lat, lon) float32 2GB nan nan nan nan ... nan nan nan nan
    nobs        (time, lat, lon) float32 2GB nan nan nan nan ... nan nan nan nan
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [12]:
ds = ds.rename({'lat': 'latitude', 'lon': 'longitude', 'time': 'date'})

In [13]:
import xarray as xr
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from typing import Optional, List
from datetime import datetime

def visualize_variable_on_map(
    dataset: xr.Dataset,
    variable: str,
    time_dim: str = 'date',
    lat_dim: str = 'latitude',
    lon_dim: str = 'longitude',
    plot_type: str = 'scatter_geo',  # Options: 'scatter_geo', 'imshow'
    downsample_factor: Optional[int] = 1,
    projection: str = 'natural earth',
    color_scale: str = 'Viridis',
    title: Optional[str] = None,
    animation_frame: Optional[str] = None,
    hover_precision: int = 2,
    custom_colorbar_title: Optional[str] = None,
    start_date: Optional[str] = None,  # New parameter for start date
    end_date: Optional[str] = None,    # New parameter for end date
    **kwargs
) -> go.Figure:
    """
    Universal function to visualize any variable from an xarray Dataset on a map with optional animation using Plotly.
    
    Parameters:
    - dataset (xr.Dataset): The xarray Dataset containing the data.
    - variable (str): The name of the variable to visualize.
    - time_dim (str): Name of the time dimension. Default is 'time'.
    - lat_dim (str): Name of the latitude dimension. Default is 'lat'.
    - lon_dim (str): Name of the longitude dimension. Default is 'lon'.
    - plot_type (str): Type of plot. Options include 'scatter_geo' and 'imshow'. Default is 'scatter_geo'.
    - downsample_factor (int, optional): Factor by which to downsample spatial data for performance. Default is 1 (no downsampling).
    - projection (str): Map projection style for Plotly. Default is 'natural earth'.
    - color_scale (str or list): Color scale for the plot. Default is 'Viridis'.
    - title (str, optional): Title of the plot. If None, a default title is generated.
    - animation_frame (str, optional): Dimension to animate over. If None, no animation is created. Default is None.
    - hover_precision (int): Decimal precision for hover data. Default is 2.
    - custom_colorbar_title (str, optional): Custom title for the color bar. If None, the variable name is used.
    - start_date (str, optional): Start date for the data in ISO format (YYYY-MM-DD). If None, no start date filtering is applied.
    - end_date (str, optional): End date for the data in ISO format (YYYY-MM-DD). If None, no end date filtering is applied.
    - **kwargs: Additional keyword arguments passed to the Plotly plotting functions.
    
    Returns:
    - fig (plotly.graph_objs._figure.Figure): The Plotly figure object.
    """
    
    # Validate plot_type
    valid_plot_types = ['scatter_geo', 'imshow']
    if plot_type not in valid_plot_types:
        raise ValueError(f"Invalid plot_type '{plot_type}'. Valid options are: {valid_plot_types}")
    
    # Check if the variable exists
    if variable not in dataset.data_vars:
        raise ValueError(f"Variable '{variable}' not found in the dataset.")
    
    # Extract the DataArray
    da = dataset[variable]
    
    # Ensure required dimensions are present
    for dim in [time_dim, lat_dim, lon_dim]:
        if dim not in da.dims:
            raise ValueError(f"Dimension '{dim}' not found in variable '{variable}'.")
    
    # Convert time dimension to datetime if not already
    if not pd.api.types.is_datetime64_any_dtype(da[time_dim].dtype):
        da[time_dim] = pd.to_datetime(da[time_dim].values)
    
    # Apply date range filtering if specified
    if start_date or end_date:
        if start_date:
            try:
                start_datetime = pd.to_datetime(start_date)
                da = da.sel({time_dim: da[time_dim] >= start_datetime})
            except Exception as e:
                raise ValueError(f"Invalid start_date '{start_date}': {e}")
        if end_date:
            try:
                end_datetime = pd.to_datetime(end_date)
                da = da.sel({time_dim: da[time_dim] <= end_datetime})
            except Exception as e:
                raise ValueError(f"Invalid end_date '{end_date}': {e}")
    
    # Check if the dataset is empty after filtering
    if da.date.size == 0:
        raise ValueError("No data available for the specified date range.")
    
    # Downsample if required
    if downsample_factor > 1:
        da = da.coarsen({lat_dim: downsample_factor, lon_dim: downsample_factor}, boundary='trim').mean()
    
    # Handle animation
    animate = False
    if animation_frame and animation_frame in da.dims:
        animate = True
    elif animation_frame:
        raise ValueError(f"Animation frame dimension '{animation_frame}' not found in variable '{variable}'.")
    
    # Generate title if not provided
    if not title:
        if start_date and end_date:
            title = f"{variable} from {start_date} to {end_date}"
        elif start_date:
            title = f"{variable} from {start_date} onwards"
        elif end_date:
            title = f"{variable} up to {end_date}"
        else:
            title = f"{variable} Visualization"
    
    if plot_type == 'scatter_geo':
        # Convert to DataFrame
        df = da.to_dataframe().reset_index()
    
        # Drop NaNs
        df = df.dropna(subset=[variable])
    
        # Convert animation frame to string if animating
        if animate:
            df[animation_frame] = df[animation_frame].astype(str)
        else:
            # If not animating, set a default frame
            df['Frame'] = 'Frame'
    
        # Create the scatter_geo plot
        fig = px.scatter_geo(
            df,
            lat=lat_dim,
            lon=lon_dim,
            color=variable,
            animation_frame=animation_frame if animate else 'Frame',
            projection=projection,
            color_continuous_scale=color_scale,
            title=title,
            labels={variable: variable.upper()},
            hover_data={variable: f':.{hover_precision}f'},
            **kwargs
        )
    
        # Update layout for better appearance
        fig.update_layout(
            coloraxis_colorbar=dict(
                title=custom_colorbar_title if custom_colorbar_title else variable.upper(),
                ticks="outside"
            )
        )
    
        # If not animating, remove animation controls
        if not animate:
            fig.update_layout(
                updatemenus=[]
            )
    
    elif plot_type == 'imshow':
        # For imshow, we need to handle each frame separately
        if animate:
            # Prepare frames
            frames = []
            times = da[animation_frame].values
            for t in times:
                da_t = da.sel({animation_frame: t})
                # Convert to numpy array
                z = da_t.values
                # Handle latitude descending for proper display
                if da[lat_dim][0] > da[lat_dim][-1]:
                    z = z[::-1, :]
                frames.append(go.Frame(
                    data=[go.Heatmap(
                        z=z,
                        x=da[lon_dim].values,
                        y=da[lat_dim].values,
                        colorscale=color_scale,
                        **kwargs
                    )],
                    name=str(t)
                ))
    
            # Initial frame
            initial_t = times[0]
            initial_da = da.sel({animation_frame: initial_t})
            z_init = initial_da.values
            if da[lat_dim][0] > da[lat_dim][-1]:
                z_init = z_init[::-1, :]
    
            # Create initial heatmap
            fig = go.Figure(
                data=go.Heatmap(
                    z=z_init,
                    x=da[lon_dim].values,
                    y=da[lat_dim].values,
                    colorscale=color_scale,
                    **kwargs
                ),
                layout=go.Layout(
                    title=title,
                    updatemenus=[dict(
                        type='buttons',
                        buttons=[
                            dict(label='Play',
                                 method='animate',
                                 args=[None, {"frame": {"duration": 500, "redraw": True},
                                              "fromcurrent": True}]),
                            dict(label='Pause',
                                 method='animate',
                                 args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                "mode": "immediate",
                                                "transition": {"duration": 0}}])
                        ],
                        showactive=False,
                        x=0.1,
                        y=0
                    )]
                ),
                frames=frames
            )
    
            # Update color bar
            fig.update_traces(colorbar=dict(
                title=custom_colorbar_title if custom_colorbar_title else variable.upper(),
                ticks="outside"
            ))
    
        else:
            # Single heatmap without animation
            da_t = da
            z = da_t.values
            if da[lat_dim][0] > da[lat_dim][-1]:
                z = z[::-1, :]
    
            fig = px.imshow(
                z,
                labels=dict(x=lon_dim, y=lat_dim, color=variable),
                x=da[lon_dim].values,
                y=da[lat_dim].values,
                color_continuous_scale=color_scale,
                title=title,
                **kwargs
            )
    
            # Update color bar
            fig.update_layout(
                coloraxis_colorbar=dict(
                    title=custom_colorbar_title if custom_colorbar_title else variable.upper(),
                    ticks="outside"
                )
            )
    
    else:
        raise NotImplementedError(f"Plot type '{plot_type}' is not implemented.")
    
    return fig




In [14]:
# # Example Usage:
# # Assuming you have your xarray Dataset loaded as `ds`

# # Visualize 'sm' with animation using scatter_geo and a specified date range
# fig = visualize_variable_on_map(
#     dataset=ds,
#     variable='sm',
#     plot_type='scatter_geo',
#     animation_frame='date',
#     downsample_factor=5,
#     title='Soil Moisture Over Time',
#     start_date='2020-01-01',  # Specify the start date
#     end_date='2020-01-31',    # Specify the end date
#     color_scale = [
#         [0.0, "darkred"],        # 0% - extremely dry
#         [0.1, "red"],            # 10% - very dry
#         [0.2, "orangered"],      # 20% - dry
#         [0.3, "lightgreen"],     # 30% - beginning of optimal moisture
#         [0.4, "limegreen"],      # 40% - optimal moisture for most crops
#         [0.5, "green"],          # 50% - middle of optimal moisture
#         [0.55, "darkseagreen"],  # 55% - upper limit of optimal moisture
#         [0.6, "darkgreen"],      # 60% - end of optimal moisture
#         [0.7, "lightblue"],      # 70% - moist but not excessive
#         [0.8, "skyblue"],        # 80% - very moist
#         [0.9, "deepskyblue"],    # 90% - extremely moist
#         [1.0, "blue"]            # 100% - very wet
#     ]
# )
# fig.show()

In [15]:
ds

<xarray.Dataset> Size: 8GB
Dimensions:     (date: 376, latitude: 720, longitude: 1440)
Coordinates:
  * latitude    (latitude) float32 3kB 89.88 89.62 89.38 ... -89.62 -89.88
  * longitude   (longitude) float32 6kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * date        (date) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
Data variables:
    sm          (date, latitude, longitude) float32 2GB nan nan nan ... nan nan
    sensor      (date, latitude, longitude) float64 3GB nan nan nan ... nan nan
    freqbandID  (date, latitude, longitude) float32 2GB nan nan nan ... nan nan
    nobs        (date, latitude, longitude) float32 2GB nan nan nan ... nan nan
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [16]:
ds

<xarray.Dataset> Size: 8GB
Dimensions:     (date: 376, latitude: 720, longitude: 1440)
Coordinates:
  * latitude    (latitude) float32 3kB 89.88 89.62 89.38 ... -89.62 -89.88
  * longitude   (longitude) float32 6kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * date        (date) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
Data variables:
    sm          (date, latitude, longitude) float32 2GB nan nan nan ... nan nan
    sensor      (date, latitude, longitude) float64 3GB nan nan nan ... nan nan
    freqbandID  (date, latitude, longitude) float32 2GB nan nan nan ... nan nan
    nobs        (date, latitude, longitude) float32 2GB nan nan nan ... nan nan
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [17]:
import numpy as np
import xarray as xr

# Step 1: Define target grid with 1° resolution
target_lat = np.arange(-30, 30.1, 1)   # From -90 to 90 degrees inclusive
target_lon = np.arange(-180, 180, 1)      # From 0 to 359.5 degrees inclusive

# Step 2: Create target grid Dataset (optional, for reference)
target_grid = xr.Dataset(
    {
        "latitude": (["latitude"], target_lat),
        "longitude": (["longitude"], target_lon),
    }
)

# Step 3: Ensure latitude is ascending
if ds.latitude[0] > ds.latitude[-1]:
    ds = ds.sortby("latitude")

# Step 4: Perform interpolation
ds = ds.interp(latitude=target_lat, longitude=target_lon, method="linear")

In [18]:
ds

<xarray.Dataset> Size: 264MB
Dimensions:     (date: 376, latitude: 61, longitude: 360)
Coordinates:
  * date        (date) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
  * latitude    (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude   (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Data variables:
    sm          (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    sensor      (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    freqbandID  (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    nobs        (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [19]:
sm = ds

In [20]:
import xarray as xr 

ds = xr.open_dataset('/teamspace/studios/this_studio/63cb306add675f5358e785f23dffe828.nc')
# ds = xr.open_dataset('/Users/adamprzychodni/Documents/Repos/ml-drought-forecasting/ml-modeling-pipeline/data/01_raw/e16fe5f664d5290665a25e5afad51ce9.nc')

In [21]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:    (date: 276, latitude: 241, longitude: 1440)
Coordinates:
    number     int64 8B ...
  * date       (date) int64 2kB 20000101 20000201 20000301 ... 20221101 20221201
  * latitude   (latitude) float64 2kB 30.0 29.75 29.5 ... -29.5 -29.75 -30.0
  * longitude  (longitude) float64 12kB -180.0 -179.8 -179.5 ... 179.5 179.8
    expver     (date) <U4 4kB ...
Data variables:
    u10        (date, latitude, longitude) float32 383MB ...
    v10        (date, latitude, longitude) float32 383MB ...
    t2m        (date, latitude, longitude) float32 383MB ...
    sst        (date, latitude, longitude) float32 383MB ...
    sp         (date, latitude, longitude) float32 383MB ...
    tp         (date, latitude, longitude) float32 383MB ...
    skt        (date, latitude, longitude) float32 383MB ...
    tcc        (date, latitude, longitude) float32 383MB ...
    e          (date, latitude, longitude) float32 383MB ...
    pev        (date, latitude, longitude) float32 383MB ...
    stl1       (date, latitude, longitude) float32 383MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-13T13:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [22]:
import numpy as np
import xarray as xr

# Step 1: Define target grid with 1° resolution
target_lat = np.arange(-30, 30.1, 1)   # From -90 to 90 degrees inclusive
target_lon = np.arange(-180, 180, 1)      # From 0 to 359.5 degrees inclusive

# Step 2: Create target grid Dataset (optional, for reference)
target_grid = xr.Dataset(
    {
        "latitude": (["latitude"], target_lat),
        "longitude": (["longitude"], target_lon),
    }
)

# Step 3: Ensure latitude is ascending
if ds.latitude[0] > ds.latitude[-1]:
    ds = ds.sortby("latitude")

# Step 4: Perform interpolation
ds = ds.interp(latitude=target_lat, longitude=target_lon, method="linear")


In [23]:
ds

<xarray.Dataset> Size: 533MB
Dimensions:    (date: 276, latitude: 61, longitude: 360)
Coordinates:
    number     int64 8B 0
  * date       (date) int64 2kB 20000101 20000201 20000301 ... 20221101 20221201
    expver     (date) <U4 4kB '0001' '0001' '0001' ... '0001' '0001' '0001'
  * latitude   (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude  (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Data variables:
    u10        (date, latitude, longitude) float64 48MB -3.321 -3.517 ... 2.212
    v10        (date, latitude, longitude) float64 48MB -0.07545 ... -1.886
    t2m        (date, latitude, longitude) float64 48MB 294.8 294.7 ... 293.2
    sst        (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.2
    sp         (date, latitude, longitude) float64 48MB 1.012e+05 ... 1.015e+05
    tp         (date, latitude, longitude) float64 48MB 0.003505 ... 0.004789
    skt        (date, latitude, longitude) float64 48MB 295.5 295.3 ... 295.0
    tcc        (date, latitude, longitude) float64 48MB 0.7003 0.6917 ... 0.6505
    e          (date, latitude, longitude) float64 48MB -0.002701 ... -0.007106
    pev        (date, latitude, longitude) float64 48MB 7.823e-08 ... -1.49e-08
    stl1       (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.3
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-13T13:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [24]:
import pandas as pd

# 1. Convert 'date' coordinate to datetime
ds['date'] = pd.to_datetime(ds['date'].astype(str), format='%Y%m%d')

In [25]:
ds

<xarray.Dataset> Size: 533MB
Dimensions:    (date: 276, latitude: 61, longitude: 360)
Coordinates:
    number     int64 8B 0
  * date       (date) datetime64[ns] 2kB 2000-01-01 2000-02-01 ... 2022-12-01
    expver     (date) <U4 4kB '0001' '0001' '0001' ... '0001' '0001' '0001'
  * latitude   (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude  (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Data variables:
    u10        (date, latitude, longitude) float64 48MB -3.321 -3.517 ... 2.212
    v10        (date, latitude, longitude) float64 48MB -0.07545 ... -1.886
    t2m        (date, latitude, longitude) float64 48MB 294.8 294.7 ... 293.2
    sst        (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.2
    sp         (date, latitude, longitude) float64 48MB 1.012e+05 ... 1.015e+05
    tp         (date, latitude, longitude) float64 48MB 0.003505 ... 0.004789
    skt        (date, latitude, longitude) float64 48MB 295.5 295.3 ... 295.0
    tcc        (date, latitude, longitude) float64 48MB 0.7003 0.6917 ... 0.6505
    e          (date, latitude, longitude) float64 48MB -0.002701 ... -0.007106
    pev        (date, latitude, longitude) float64 48MB 7.823e-08 ... -1.49e-08
    stl1       (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.3
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-13T13:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [26]:
sm

<xarray.Dataset> Size: 264MB
Dimensions:     (date: 376, latitude: 61, longitude: 360)
Coordinates:
  * date        (date) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
  * latitude    (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude   (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Data variables:
    sm          (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    sensor      (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    freqbandID  (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
    nobs        (date, latitude, longitude) float64 66MB nan nan nan ... nan nan
Attributes: (12/44)
    title:                        C3S Satellite Surface Soil Moisture merged ...
    institution:                  TU Wien (AUT); VanderSat B.V. (NL)
    contact:                      cci_sm_contact@eodc.eu
    source:                       WARP 5.5R1.1/AMI-WS/ERS12 Level 2 Soil Mois...
    platform:                     ERS-1, ERS-2, METOP-A, METOP-B
    processing_level:             Quality-controlled, super-collocated (L3S) ...
    ...                           ...
    id:                           C3S-SOILMOISTURE-L3S-SSMS-ACTIVE-MONTHLY-19...
    history:                      2023-06-29T13:41:20.415211 mean calculated
    date_created:                 2023-06-29T13:41:20Z
    time_coverage_duration:       P1M
    time_coverage_start:          1991-08-31T12:00:00Z
    time_coverage_end:            1991-09-30T12:00:00Z

In [27]:
ds_merged = xr.merge([ds, sm])


In [28]:
ds_merged = ds_merged.fillna(0)

In [29]:
ds_merged

<xarray.Dataset> Size: 991MB
Dimensions:     (date: 376, latitude: 61, longitude: 360)
Coordinates:
  * date        (date) datetime64[ns] 3kB 1991-09-01 1991-10-01 ... 2022-12-01
  * latitude    (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude   (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
    number      int64 8B 0
    expver      (date) object 3kB nan nan nan nan ... '0001' '0001' '0001'
Data variables: (12/15)
    u10         (date, latitude, longitude) float64 66MB 0.0 0.0 ... 2.05 2.212
    v10         (date, latitude, longitude) float64 66MB 0.0 0.0 ... -1.886
    t2m         (date, latitude, longitude) float64 66MB 0.0 0.0 ... 293.1 293.2
    sst         (date, latitude, longitude) float64 66MB 0.0 0.0 ... 295.1 295.2
    sp          (date, latitude, longitude) float64 66MB 0.0 0.0 ... 1.015e+05
    tp          (date, latitude, longitude) float64 66MB 0.0 0.0 ... 0.004789
    ...          ...
    pev         (date, latitude, longitude) float64 66MB 0.0 0.0 ... -1.49e-08
    stl1        (date, latitude, longitude) float64 66MB 0.0 0.0 ... 295.1 295.3
    sm          (date, latitude, longitude) float64 66MB 0.0 0.0 0.0 ... 0.0 0.0
    sensor      (date, latitude, longitude) float64 66MB 0.0 0.0 0.0 ... 0.0 0.0
    freqbandID  (date, latitude, longitude) float64 66MB 0.0 0.0 0.0 ... 0.0 0.0
    nobs        (date, latitude, longitude) float64 66MB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-13T13:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [30]:
# Convert the dates to strings to filter
start_date = '2000-01-01 '
end_date = '2022-12-01'

# Select the date range in the dataset
ds = ds_merged.sel(date=slice(start_date, end_date))

In [31]:
ds

<xarray.Dataset> Size: 727MB
Dimensions:     (date: 276, latitude: 61, longitude: 360)
Coordinates:
  * date        (date) datetime64[ns] 2kB 2000-01-01 2000-02-01 ... 2022-12-01
  * latitude    (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude   (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
    number      int64 8B 0
    expver      (date) object 2kB '0001' '0001' '0001' ... '0001' '0001' '0001'
Data variables: (12/15)
    u10         (date, latitude, longitude) float64 48MB -3.321 -3.517 ... 2.212
    v10         (date, latitude, longitude) float64 48MB -0.07545 ... -1.886
    t2m         (date, latitude, longitude) float64 48MB 294.8 294.7 ... 293.2
    sst         (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.2
    sp          (date, latitude, longitude) float64 48MB 1.012e+05 ... 1.015e+05
    tp          (date, latitude, longitude) float64 48MB 0.003505 ... 0.004789
    ...          ...
    pev         (date, latitude, longitude) float64 48MB 7.823e-08 ... -1.49e-08
    stl1        (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.3
    sm          (date, latitude, longitude) float64 48MB 0.0 0.0 0.0 ... 0.0 0.0
    sensor      (date, latitude, longitude) float64 48MB 0.0 0.0 0.0 ... 0.0 0.0
    freqbandID  (date, latitude, longitude) float64 48MB 0.0 0.0 0.0 ... 0.0 0.0
    nobs        (date, latitude, longitude) float64 48MB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-13T13:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [32]:
ds = ds.drop_vars(['sensor', 'freqbandID', 'nobs'])


In [33]:
# ds = ds.fillna(0)

In [34]:
ds

<xarray.Dataset> Size: 582MB
Dimensions:    (date: 276, latitude: 61, longitude: 360)
Coordinates:
  * date       (date) datetime64[ns] 2kB 2000-01-01 2000-02-01 ... 2022-12-01
  * latitude   (latitude) float64 488B -30.0 -29.0 -28.0 ... 28.0 29.0 30.0
  * longitude  (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
    number     int64 8B 0
    expver     (date) object 2kB '0001' '0001' '0001' ... '0001' '0001' '0001'
Data variables:
    u10        (date, latitude, longitude) float64 48MB -3.321 -3.517 ... 2.212
    v10        (date, latitude, longitude) float64 48MB -0.07545 ... -1.886
    t2m        (date, latitude, longitude) float64 48MB 294.8 294.7 ... 293.2
    sst        (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.2
    sp         (date, latitude, longitude) float64 48MB 1.012e+05 ... 1.015e+05
    tp         (date, latitude, longitude) float64 48MB 0.003505 ... 0.004789
    skt        (date, latitude, longitude) float64 48MB 295.5 295.3 ... 295.0
    tcc        (date, latitude, longitude) float64 48MB 0.7003 0.6917 ... 0.6505
    e          (date, latitude, longitude) float64 48MB -0.002701 ... -0.007106
    pev        (date, latitude, longitude) float64 48MB 7.823e-08 ... -1.49e-08
    stl1       (date, latitude, longitude) float64 48MB 295.6 295.5 ... 295.3
    sm         (date, latitude, longitude) float64 48MB 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-13T13:04 GRIB to CDM+CF via cfgrib-0.9.1...

In [35]:
# # Example Usage:
# # Assuming you have your xarray Dataset loaded as `ds`

# # Visualize 'sm' with animation using scatter_geo and a specified date range
# fig = visualize_variable_on_map(
#     dataset=ds,
#     variable='sm',
#     plot_type='scatter_geo',
#     animation_frame='date',
#     downsample_factor=5,
#     title='Soil Moisture Over Time',
#     start_date='2020-01-01',  # Specify the start date
#     end_date='2020-01-31',    # Specify the end date
#     color_scale = [
#         [0.0, "darkred"],        # 0% - extremely dry
#         [0.1, "red"],            # 10% - very dry
#         [0.2, "orangered"],      # 20% - dry
#         [0.3, "lightgreen"],     # 30% - beginning of optimal moisture
#         [0.4, "limegreen"],      # 40% - optimal moisture for most crops
#         [0.5, "green"],          # 50% - middle of optimal moisture
#         [0.55, "darkseagreen"],  # 55% - upper limit of optimal moisture
#         [0.6, "darkgreen"],      # 60% - end of optimal moisture
#         [0.7, "lightblue"],      # 70% - moist but not excessive
#         [0.8, "skyblue"],        # 80% - very moist
#         [0.9, "deepskyblue"],    # 90% - extremely moist
#         [1.0, "blue"]            # 100% - very wet
#     ]
# )
# fig.show()

In [36]:
import numpy as np
import xarray as xr

# Assuming your dataset is called ds
# Example dataset variables: t2m, sst, tp, pev

# 1. Extract relevant variables
targets = ['sm']
data_arrays = [ds[var] for var in targets]

# 2. Flatten latitude and longitude into a single node dimension
# Combine the latitude and longitude as one "node" dimension
data_arrays_flattened = [da.stack(node=('latitude', 'longitude')) for da in data_arrays]

# 3. Convert each variable's DataArray to a numpy array and add a new channel dimension
# (so we have the shape (time, nodes, channels))
target = np.stack([da.to_numpy() for da in data_arrays_flattened], axis=-1)


In [37]:
target

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]])

In [38]:
# Save as .npy file
np.save('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/target.npy', target)

In [39]:

mask = np.where(~np.isnan(target), 1, 0)

In [40]:
mask

array([[[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       ...,

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]],

       [[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]]])

In [41]:
# Save as .npy file
np.save('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/mask.npy', mask)

In [42]:
import numpy as np
import xarray as xr

# Assuming your dataset is called ds
# Example dataset variables: t2m, sst, tp, pev

# 1. Extract relevant variables
variables = ['u10', 'v10', 't2m', 'sst', 'sp', 'tp', 'skt', 'tcc', 'e', 'pev', 'stl1']

data_arrays = [ds[var] for var in variables]

# 2. Flatten latitude and longitude into a single node dimension
# Combine the latitude and longitude as one "node" dimension
data_arrays_flattened = [da.stack(node=('latitude', 'longitude')) for da in data_arrays]

# 3. Convert each variable's DataArray to a numpy array and add a new channel dimension
# (so we have the shape (time, nodes, channels))
covariates = np.stack([da.to_numpy() for da in data_arrays_flattened], axis=-1)


In [43]:
covariates

array([[[-3.32063580e+00, -7.54508972e-02,  2.94796997e+02, ...,
         -2.70088762e-03,  7.82310963e-08,  2.95601257e+02],
        [-3.51741314e+00, -1.53087616e-01,  2.94720825e+02, ...,
         -2.74475664e-03,  7.82310963e-08,  2.95468445e+02],
        [-3.38997173e+00, -5.78727722e-02,  2.94691528e+02, ...,
         -2.67370790e-03,  7.82310963e-08,  2.95267273e+02],
        ...,
        [ 1.06071186e+00,  1.03929520e+00,  2.91923950e+02, ...,
         -4.93439287e-03,  7.82310963e-08,  2.93827820e+02],
        [ 1.08903217e+00,  1.23851395e+00,  2.92054810e+02, ...,
         -4.63636965e-03,  7.82310963e-08,  2.93753601e+02],
        [ 1.01139545e+00,  1.40111160e+00,  2.92066528e+02, ...,
         -4.19386476e-03,  7.82310963e-08,  2.93562195e+02]],

       [[-3.16875172e+00,  1.78701401e+00,  2.95360138e+02, ...,
         -3.15690413e-03, -2.60770321e-08,  2.96407532e+02],
        [-2.94609547e+00,  1.47256088e+00,  2.95342560e+02, ...,
         -2.75802985e-03, -2.60770321e

In [44]:
import numpy as np

# Zakładając, że 'covariates' jest Twoim array'em numpy
covariates = np.nan_to_num(covariates, nan=0.0)

In [45]:
covariates


array([[[-3.32063580e+00, -7.54508972e-02,  2.94796997e+02, ...,
         -2.70088762e-03,  7.82310963e-08,  2.95601257e+02],
        [-3.51741314e+00, -1.53087616e-01,  2.94720825e+02, ...,
         -2.74475664e-03,  7.82310963e-08,  2.95468445e+02],
        [-3.38997173e+00, -5.78727722e-02,  2.94691528e+02, ...,
         -2.67370790e-03,  7.82310963e-08,  2.95267273e+02],
        ...,
        [ 1.06071186e+00,  1.03929520e+00,  2.91923950e+02, ...,
         -4.93439287e-03,  7.82310963e-08,  2.93827820e+02],
        [ 1.08903217e+00,  1.23851395e+00,  2.92054810e+02, ...,
         -4.63636965e-03,  7.82310963e-08,  2.93753601e+02],
        [ 1.01139545e+00,  1.40111160e+00,  2.92066528e+02, ...,
         -4.19386476e-03,  7.82310963e-08,  2.93562195e+02]],

       [[-3.16875172e+00,  1.78701401e+00,  2.95360138e+02, ...,
         -3.15690413e-03, -2.60770321e-08,  2.96407532e+02],
        [-2.94609547e+00,  1.47256088e+00,  2.95342560e+02, ...,
         -2.75802985e-03, -2.60770321e

In [46]:
# Save as .npy file
np.save('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/covariates.npy', covariates)

In [47]:
import pandas as pd
import os

def generate_and_save_metadata(df: pd.DataFrame, lat_col: str = 'lat', lon_col: str = 'lon', save_directory: str = None) -> pd.DataFrame:
    """
    Generates metadata from a given DataFrame by identifying unique latitude and longitude
    combinations and assigning a unique node ID to each combination. The metadata is then
    saved to a specified Parquet file.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - lat_col (str): The name of the column containing latitude data.
    - lon_col (str): The name of the column containing longitude data.
    - save_directory (str, optional): The directory where the metadata file will be saved. If None, 
                                      the file will be saved in the current working directory.

    Returns:
    - pd.DataFrame: A DataFrame containing the metadata with latitude, longitude, and 'node_id' as columns.
    """
    # Ensure the DataFrame contains necessary columns
    if lat_col not in df.columns or lon_col not in df.columns:
        raise ValueError(f"DataFrame must contain '{lat_col}' and '{lon_col}' columns.")

    # Create a unique node ID for each unique latitude-longitude combination
    unique_lat_lon = df[[lat_col, lon_col]].drop_duplicates().reset_index(drop=True)
    unique_lat_lon['node_id'] = unique_lat_lon.index

    # Create the metadata DataFrame
    metadata = unique_lat_lon.set_index('node_id')

    # Handle save directory and save metadata to Parquet file
    if save_directory:
        os.makedirs(save_directory, exist_ok=True)  # Create directory if it doesn't exist
        file_path = os.path.join(save_directory, "metadata.parquet")
    else:
        file_path = "metadata.parquet"  # Save in the current working directory

    metadata.to_parquet(file_path)
    print(f"Metadata file saved at: {file_path}")

    return metadata


In [48]:
import pandas as pd
import os

# Assume you've already stacked your data arrays as 'data_arrays_flattened'
# We'll use one of the flattened DataArrays to extract the node information
da_flattened = data_arrays_flattened[0]  # Using the first variable for example

# Get the MultiIndex from the 'node' dimension
node_index = da_flattened.indexes['node']

# Extract latitude and longitude from the MultiIndex
latitudes = node_index.get_level_values('latitude').values
longitudes = node_index.get_level_values('longitude').values

# Create a DataFrame with 'lat' and 'lon' columns
df = pd.DataFrame({
    'lat': latitudes,
    'lon': longitudes
})

# Now use your 'generate_and_save_metadata' function
metadata = generate_and_save_metadata(
    df, 
    lat_col='lat', 
    lon_col='lon', 
    save_directory='ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/'
)


Metadata file saved at: ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.parquet


<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject


In [49]:
# metadata

In [50]:
import pandas as pd
metadata = pd.read_parquet("ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.parquet")

In [51]:
metadata

,lat,lon
node_id,,
0,-30.0,-180
1,-30.0,-179
2,-30.0,-178
3,-30.0,-177
4,-30.0,-176
...,...,...
21955,30.0,175
21956,30.0,176
21957,30.0,177


In [52]:
# Converting the DataFrame to a numpy ndarray
metadata_array = metadata.to_numpy()

In [53]:
metadata_array

array([[ -30., -180.],
       [ -30., -179.],
       [ -30., -178.],
       ...,
       [  30.,  177.],
       [  30.,  178.],
       [  30.,  179.]])

In [54]:
# Save as .npy file
np.save('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.npy', metadata_array)

In [55]:
from tsl.ops.similarities import geographical_distance
# Calculate geographical distances with coordinates converted to radians.
dist = geographical_distance(metadata_array, to_rad=True)

In [56]:
dist

array([[   0.        ,   96.29745869,  192.5930838 , ..., 6679.26370783,
        6675.06538725, 6672.54511721],
       [  96.29745869,    0.        ,   96.29745869, ..., 6685.13689318,
        6679.26370783, 6675.06538725],
       [ 192.5930838 ,   96.29745869,    0.        , ..., 6692.68049995,
        6685.13689318, 6679.26370783],
       ...,
       [6679.26370783, 6685.13689318, 6692.68049995, ...,    0.        ,
          96.29745869,  192.5930838 ],
       [6675.06538725, 6679.26370783, 6685.13689318, ...,   96.29745869,
           0.        ,   96.29745869],
       [6672.54511721, 6675.06538725, 6679.26370783, ...,  192.5930838 ,
          96.29745869,    0.        ]])

In [57]:
# Save as .npy file
np.save('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/distances.npy', dist)

In [58]:
from typing import Optional, Union, List
import numpy as np 

from tsl.datasets.prototypes import TabularDataset

class DroughtDataset(TabularDataset):

    similarity_options = {'distance', 'correlation'}

    def __init__(self,
                 root: str = None
                 ):

        self.root = root

        # Load data
        target, mask, u, dist, metadata = self.load()

        covariates = {
            'u': (u),
            'metadata' : (metadata),
            'distances': (dist)
        }

        super().__init__(target=target,
                         mask=mask,
                         covariates=covariates,
                         similarity_score='distance',
                         temporal_aggregation='mean',
                         spatial_aggregation='mean',
                         name='DroughtDataset')

    def load(self):
        """
        Load data from files.

        Returns:
            tuple: Containing target, mask, covariates, distances, and metadata.
        """
        target_path = f"{self.root}target.npy"
        mask_path = f"{self.root}mask.npy"
        dist_path = f"{self.root}distances.npy"
        covariates_path = f"{self.root}covariates.npy"
        metadata_path = f"{self.root}metadata.npy"

        # Load main data
        target = np.load(target_path)
        mask = np.load(mask_path)
        u = np.load(covariates_path)
        dist = np.load(dist_path)
        metadata = np.load(metadata_path)

        return target, mask, u, dist, metadata


    def compute_similarity(self, method: str, **kwargs):
        """
        Compute similarity matrix based on the specified method.

        Args:
            method (str): The similarity computation method ('distance' or 'correlation').
            **kwargs: Additional keyword arguments for similarity computation.

        Returns:
            numpy.ndarray: Computed similarity matrix.

        Raises:
            ValueError: If an unknown similarity method is provided.
        """
        if method == "distance":
            # Calculate a Gaussian kernel similarity from the distance matrix, using a default or provided 'theta'
            theta = kwargs.get('theta', np.std(self.distances))
            return self.gaussian_kernel(self.distances, theta=theta)
        elif method == "correlation":
            # Compute the average correlation between nodes over the target features
            # Reshape target data to have nodes as columns
            target_values = self.target.values.reshape(len(self.target), -1, len(self.target_node_feature))
            # Average over the target features
            target_mean = target_values.mean(axis=2)
            # Compute correlation between nodes
            corr = np.corrcoef(target_mean, rowvar=False)
            return (corr + 1) / 2  # Normalize to [0, 1]
        else:
            raise ValueError(f"Unknown similarity method: {method}")

    @staticmethod
    def gaussian_kernel(distances, theta):
        """
        Compute Gaussian kernel similarity from distances.

        Args:
            distances (numpy.ndarray): Distance matrix.
            theta (float): Kernel bandwidth parameter.

        Returns:
            numpy.ndarray: Gaussian kernel similarity matrix.
        """
        return np.exp(-(distances ** 2) / (2 * (theta ** 2)))

In [59]:
dataset = DroughtDataset(root='ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/')

In [60]:
dataset.target

array([[[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       ...,

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]],

       [[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]], dtype=float32)

In [61]:
print(f"Has missing values: {dataset.has_mask}")

Has missing values: True


In [62]:
dataset.mask

array([[[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [ True],
        [ True]],

       [[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [ True],
        [ True]],

       [[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [ True],
        [ True]],

       ...,

       [[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [ True],
        [ True]],

       [[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [ True],
        [ True]],

       [[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [ True],
        [ True]]])

In [63]:
dataset.set_mask(dataset.mask)

In [64]:
dataset.covariates

{'u': array([[[-3.3206358e+00, -7.5450897e-02,  2.9479700e+02, ...,
          -2.7008876e-03,  7.8231096e-08,  2.9560126e+02],
         [-3.5174131e+00, -1.5308762e-01,  2.9472083e+02, ...,
          -2.7447566e-03,  7.8231096e-08,  2.9546844e+02],
         [-3.3899717e+00, -5.7872772e-02,  2.9469153e+02, ...,
          -2.6737079e-03,  7.8231096e-08,  2.9526727e+02],
         ...,
         [ 1.0607119e+00,  1.0392952e+00,  2.9192395e+02, ...,
          -4.9343929e-03,  7.8231096e-08,  2.9382782e+02],
         [ 1.0890322e+00,  1.2385139e+00,  2.9205481e+02, ...,
          -4.6363696e-03,  7.8231096e-08,  2.9375360e+02],
         [ 1.0113955e+00,  1.4011116e+00,  2.9206653e+02, ...,
          -4.1938648e-03,  7.8231096e-08,  2.9356219e+02]],
 
        [[-3.1687517e+00,  1.7870140e+00,  2.9536014e+02, ...,
          -3.1569041e-03, -2.6077032e-08,  2.9640753e+02],
         [-2.9460955e+00,  1.4725609e+00,  2.9534256e+02, ...,
          -2.7580298e-03, -2.6077032e-08,  2.9620441e+02],
  

In [65]:
dataset.distances

array([[   0.     ,   96.29746,  192.59308, ..., 6679.2637 , 6675.0654 ,
        6672.545  ],
       [  96.29746,    0.     ,   96.29746, ..., 6685.1367 , 6679.2637 ,
        6675.0654 ],
       [ 192.59308,   96.29746,    0.     , ..., 6692.6807 , 6685.1367 ,
        6679.2637 ],
       ...,
       [6679.2637 , 6685.1367 , 6692.6807 , ...,    0.     ,   96.29746,
         192.59308],
       [6675.0654 , 6679.2637 , 6685.1367 , ...,   96.29746,    0.     ,
          96.29746],
       [6672.545  , 6675.0654 , 6679.2637 , ...,  192.59308,   96.29746,
           0.     ]], dtype=float32)

In [66]:
sim = dataset.compute_similarity("distance")  # or dataset.compute_similarity()

In [67]:
sim

array([[1.        , 0.9998158 , 0.9992636 , ..., 0.4123332 , 0.41279253,
        0.41306838],
       [0.9998158 , 1.        , 0.9998158 , ..., 0.411691  , 0.4123332 ,
        0.41279253],
       [0.9992636 , 0.9998158 , 1.        , ..., 0.41086674, 0.411691  ,
        0.4123332 ],
       ...,
       [0.4123332 , 0.411691  , 0.41086674, ..., 1.        , 0.9998158 ,
        0.9992636 ],
       [0.41279253, 0.4123332 , 0.411691  , ..., 0.9998158 , 1.        ,
        0.9998158 ],
       [0.41306838, 0.41279253, 0.4123332 , ..., 0.9992636 , 0.9998158 ,
        1.        ]], dtype=float32)

In [68]:
# Adjust connectivity to reduce the number of edges
connectivity = dataset.get_connectivity(          
    knn=5,     
    include_self=False,
    normalize_axis=1,
    layout="edge_index"
)

In [69]:
edge_index, edge_weight = connectivity

In [70]:
edge_index

array([[    0,     0,     0, ..., 21959, 21959, 21959],
       [    1,   359,   360, ..., 21599, 21600, 21958]])

In [71]:
edge_weight

array([0.20002224, 0.20002224, 0.20000282, ..., 0.20000282, 0.20002224,
       0.20002224], dtype=float32)

In [72]:
def print_matrix(matrix):
    return pd.DataFrame(matrix)

In [73]:
import pandas as pd
from tsl.ops.connectivity import edge_index_to_adj

adj = edge_index_to_adj(edge_index, edge_weight)
print(f'A {adj.shape}:')
print_matrix(adj)

A (21960, 21960):


,0,1,2,3,4,5,6,7,8,9,...,21950,21951,21952,21953,21954,21955,21956,21957,21958,21959
0,0.000000,0.200022,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.200022,0.000000,0.200022,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.200022,0.000000,0.200022,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.200022,0.000000,0.200022,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.200022,0.000000,0.200022,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21955,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.200022,0.000000,0.200022,0.000000,0.000000,0.000000
21956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.200022,0.000000,0.200022,0.000000,0.000000
21957,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.200022,0.000000,0.200022,0.000000
21958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.200022,0.000000,0.200022


In [74]:
from tsl.data import SpatioTemporalDataset

# covariates=dict(u=dataset.covariates['u'])
covariates=dataset.covariates
mask = dataset.mask

torch_dataset = SpatioTemporalDataset(target=dataset.dataframe(),
                                      mask=mask,
                                      covariates=covariates,
                                      connectivity=connectivity,
                                      horizon=6, # Predict 7 step ahead
                                      window=12, # Use 30 timestamps to predict the next one
                                      stride=1 # Move 7 step forward each time
                                      )

In [75]:
torch_dataset.mask

tensor([[[True],
         [True],
         [True],
         ...,
         [True],
         [True],
         [True]],

        [[True],
         [True],
         [True],
         ...,
         [True],
         [True],
         [True]],

        [[True],
         [True],
         [True],
         ...,
         [True],
         [True],
         [True]],

        ...,

        [[True],
         [True],
         [True],
         ...,
         [True],
         [True],
         [True]],

        [[True],
         [True],
         [True],
         ...,
         [True],
         [True],
         [True]],

        [[True],
         [True],
         [True],
         ...,
         [True],
         [True],
         [True]]])

In [76]:
# torch_dataset.update_input_map(input_mask=['mask'])

In [77]:
from tsl.data.preprocessing import StandardScaler, MinMaxScaler

scalers = {
    'target': MinMaxScaler(axis=(0, 1)),
    'u': MinMaxScaler(axis=(0, 1))
}

In [78]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
                                 
# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.35, test_len=0.1)


In [79]:
# Create a SpatioTemporalDataModule
datamodule = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    mask_scaling=True,
    splitter=splitter,
    batch_size=8,
    workers=15,
    )

print(datamodule)

SpatioTemporalDataModule(train_len=None, val_len=None, test_len=None, scalers=[target, u], batch_size=8)


In [80]:
datamodule.setup()

In [81]:
datamodule

SpatioTemporalDataModule(train_len=141, val_len=69, test_len=25, scalers=[target, u], batch_size=8)

In [82]:
datamodule.splitter.indices

{'train': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140]),
 'val': array([153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
        166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
        179, 180, 181, 182, 183, 184, 185, 

In [83]:
from typing import Optional

import torch
from torch import nn, Tensor
from torch_geometric.nn import MessagePassing
from tsl.nn.blocks import RNNBase
from tsl.nn.layers import Dense, GraphGRUCellBase, Activation


class GraphAnisoConv(MessagePassing):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: int = 1,
                 edge_dim: Optional[int] = None,
                 activation: str = 'leaky_relu'):
        super(GraphAnisoConv, self).__init__(aggr="add", node_dim=-2)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        self.msg_mlps = nn.ModuleList([
            nn.Sequential(
                nn.Linear(2 * (in_channels if i == 0 else out_channels),
                          out_channels),
                Activation(activation),
                nn.Linear(out_channels, out_channels),
            )
            for i in range(kernel_size)
        ])

        edge_dim = edge_dim or 1  # accommodate for edge_weight
        self.lin_edge = nn.Linear(edge_dim, out_channels, bias=False)

        self.gate_mlp = Dense(out_channels, 1, activation='sigmoid')

        self.skip_conn = nn.Linear(in_channels, out_channels)
        self.activation = Activation(activation)

    def forward(self, x, edge_index, edge_attr: Optional[Tensor] = None):
        """"""
        out, x_ = 0, x
        for idx in range(self.kernel_size):
            x_ = self.propagate(edge_index, idx=idx, x=x_, edge_attr=edge_attr)
            out += x_
        out = self.activation(out + self.skip_conn(x))
        return out

    def message(self, x_i, x_j, idx, edge_attr: Optional[Tensor] = None):
        mij = self.msg_mlps[idx](torch.cat([x_i, x_j], -1))
        if edge_attr is not None:
            if edge_attr.ndim == 1:  # accommodate for edge_weight
                edge_attr = edge_attr.view(-1, 1)
            mij = mij + self.lin_edge(edge_attr)
        return self.gate_mlp(mij) * mij


class GraphAnisoGRUCell(GraphGRUCellBase):

    def __init__(self, input_size: int, hidden_size: int,
                 edge_dim: Optional[int] = None,
                 activation: str = 'leaky_relu'):
        self.input_size = input_size
        # instantiate gates
        forget_gate = GraphAnisoConv(input_size + hidden_size, hidden_size,
                                     edge_dim=edge_dim, activation=activation)
        update_gate = GraphAnisoConv(input_size + hidden_size, hidden_size,
                                     edge_dim=edge_dim, activation=activation)
        candidate_gate = GraphAnisoConv(input_size + hidden_size, hidden_size,
                                        edge_dim=edge_dim,
                                        activation=activation)
        super(GraphAnisoGRUCell, self).__init__(hidden_size=hidden_size,
                                                forget_gate=forget_gate,
                                                update_gate=update_gate,
                                                candidate_gate=candidate_gate)


class GraphAnisoGRU(RNNBase):

    def __init__(self, input_size: int, hidden_size: int,
                 edge_dim: Optional[int] = None,
                 n_layers: int = 1, cat_states_layers: bool = False,
                 return_only_last_state: bool = False,
                 activation: str = 'leaky_relu'):
        self.input_size = input_size
        self.hidden_size = hidden_size
        rnn_cells = [
            GraphAnisoGRUCell(input_size if i == 0 else hidden_size,
                              hidden_size, edge_dim=edge_dim,
                              activation=activation)
            for i in range(n_layers)
        ]
        super(GraphAnisoGRU, self).__init__(rnn_cells, cat_states_layers,
                                            return_only_last_state)


In [84]:
from typing import Optional, Union, List

import torch
from torch import Tensor, nn
from torch_geometric.typing import Adj
from tsl.nn.blocks import MLPDecoder
from tsl.nn.layers import MultiLinear, NodeEmbedding
from tsl.nn.models import BaseModel
from tsl.nn.utils import maybe_cat_exog
from tsl.utils import ensure_list


def maybe_cat_emb(x: Tensor, emb: Optional[Tensor]):
    if emb is None:
        return x
    if emb.ndim < x.ndim:
        emb = emb[[None] * (x.ndim - emb.ndim)]
    emb = emb.expand(*x.shape[:-1], -1)
    return torch.cat([x, emb], dim=-1)


class STGNN(BaseModel):
    available_embedding_pos = {'encoding', 'decoding'}

    def __init__(self, input_size: int, horizon: int,
                 n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Optional[
                     Union[str, List[str]]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 activation: str = 'elu'):
        super(STGNN, self).__init__()

        self.input_size = input_size
        self.horizon = horizon
        self.n_nodes = n_nodes
        self.output_size = output_size or input_size
        self.hidden_size = hidden_size
        self.exog_size = exog_size
        self.activation = activation

        # EMBEDDING
        if add_embedding_before is None:
            add_embedding_before = set()
        else:
            add_embedding_before = set(ensure_list(add_embedding_before))
            if not add_embedding_before.issubset(self.available_embedding_pos):
                raise ValueError("Parameter 'add_embedding_before' must be a "
                                 f"subset of {self.available_embedding_pos}")
        self.add_embedding_before = add_embedding_before

        if emb_size > 0:
            self.emb = NodeEmbedding(n_nodes, emb_size)
        else:
            self.register_module('emb', None)

        # ENCODER
        self.encoder_input = input_size + exog_size
        if 'encoding' in self.add_embedding_before and self.emb is not None:
            self.encoder_input += emb_size

        if use_local_weights is not None:
            self.use_local_weights = set(ensure_list(use_local_weights))
            if len(self.use_local_weights.difference(['encoder', 'decoder'])):
                raise ValueError("Parameter 'use_local_weights' must be "
                                 "'encoder', 'decoder', or both.")
        else:
            self.use_local_weights = set()

        if 'encoder' in self.use_local_weights:
            self.encoder = MultiLinear(self.encoder_input, hidden_size, n_nodes)
        else:
            self.encoder = nn.Linear(self.encoder_input, hidden_size)

        # DECODER
        self.decoder_input = hidden_size
        if 'decoding' in self.add_embedding_before and self.emb is not None:
            self.decoder_input += emb_size
        if 'decoder' in self.use_local_weights:
            raise NotImplementedError()
        else:
            self.decoder = MLPDecoder(input_size=self.decoder_input,
                                      hidden_size=self.hidden_size,
                                      output_size=self.output_size,
                                      horizon=self.horizon,
                                      activation=self.activation)

    def stmp(self, x: Tensor, edge_index: Adj,
             edge_weight: Optional[Tensor] = None,
             emb: Optional[Tensor] = None) -> Tensor:
        raise NotImplementedError

    def forward(self, x: Tensor, edge_index: Adj,
                edge_weight: Optional[Tensor] = None,
                u: Optional[Tensor] = None,
                node_idx: Optional[Tensor] = None) -> Tensor:
        """"""
        # x: [batches steps nodes features]
        x = maybe_cat_exog(x, u)
        batch_size = x.size(0)
        emb = self.emb(expand=(batch_size, -1, -1),
                       node_index=node_idx) if self.emb is not None else None

        if 'encoding' in self.add_embedding_before and emb is not None:
            x = maybe_cat_emb(x, emb[:, None])

        # ENCODER   ###########################################################
        x = self.encoder(x)

        # SPATIOTEMPORAL MESSAGE-PASSING   ####################################
        out = self.stmp(x, edge_index, edge_weight, emb)

        # DECODER   ###########################################################
        if 'decoding' in self.add_embedding_before:
            out = maybe_cat_emb(out, emb)

        out = self.decoder(out)

        return out


class TimeThenSpace(STGNN):
    available_embedding_pos = {'encoding', 'message_passing', 'decoding'}

    def __init__(self, input_size: int, horizon: int,
                 temporal_encoder: nn.Module,
                 spatial_encoder: Union[nn.Module, List[nn.Module]],
                 n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Union[str, List[str]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 activation: str = 'elu'):
        super(TimeThenSpace, self).__init__(input_size=input_size,
                                            horizon=horizon,
                                            n_nodes=n_nodes,
                                            output_size=output_size,
                                            exog_size=exog_size,
                                            hidden_size=hidden_size,
                                            emb_size=emb_size,
                                            add_embedding_before=add_embedding_before,
                                            use_local_weights=use_local_weights,
                                            activation=activation)
        # STMP
        self.temporal_encoder = temporal_encoder
        if not isinstance(spatial_encoder, nn.ModuleList):
            spatial_encoder = nn.ModuleList(ensure_list(spatial_encoder))
        self.mp_layers = spatial_encoder
        self.spatial_layers = len(self.mp_layers)

    def stmp(self, x: Tensor, edge_index: Adj,
             edge_weight: Optional[Tensor] = None,
             emb: Optional[Tensor] = None) -> Tensor:
        # temporal encoding
        out = self.temporal_encoder(x)
        # spatial encoding
        for layer in self.mp_layers:
            if 'message_passing' in self.add_embedding_before:
                out = maybe_cat_emb(out, emb)
            out = layer(out, edge_index, edge_weight)
        return out


class TimeAndSpace(STGNN):

    def __init__(self, input_size: int, horizon: int, stmp_conv: nn.Module,
                 n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Union[str, List[str]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 activation: str = 'elu'):
        super(TimeAndSpace, self).__init__(input_size=input_size,
                                           horizon=horizon,
                                           n_nodes=n_nodes,
                                           output_size=output_size,
                                           exog_size=exog_size,
                                           hidden_size=hidden_size,
                                           emb_size=emb_size,
                                           add_embedding_before=add_embedding_before,
                                           use_local_weights=use_local_weights,
                                           activation=activation)

        # STMP
        self.stmp_conv = stmp_conv

    def stmp(self, x: Tensor, edge_index: Adj,
             edge_weight: Optional[Tensor] = None,
             emb: Optional[Tensor] = None) -> Tensor:
        # spatiotemporal encoding
        out = self.stmp_conv(x, edge_index, edge_weight)
        return out


In [85]:
from typing import Union, List

class TimeAndGraphAnisoModel(TimeAndSpace):

    def __init__(self, input_size: int, horizon: int, n_nodes: int = None,
                 output_size: int = None,
                 exog_size: int = 0,
                 hidden_size: int = 32,
                 emb_size: int = 0,
                 add_embedding_before: Union[str, List[str]] = 'encoding',
                 use_local_weights: Union[str, List[str]] = None,
                 n_layers: int = 1,
                 activation: str = 'elu'):
        stmp_conv = GraphAnisoGRU(input_size=hidden_size,
                                  hidden_size=hidden_size,
                                  n_layers=n_layers,
                                  activation=activation,
                                  return_only_last_state=True)
        super(TimeAndGraphAnisoModel, self).__init__(
            input_size=input_size,
            horizon=horizon,
            stmp_conv=stmp_conv,
            n_nodes=n_nodes,
            output_size=output_size,
            exog_size=exog_size,
            hidden_size=hidden_size,
            emb_size=emb_size,
            add_embedding_before=add_embedding_before,
            use_local_weights=use_local_weights,
            activation=activation
        )


In [86]:
hidden_size = 32          # Number of hidden units
ff_size = 64             # Number of units in the feed-forward layers
n_layers = 3              # Number of SpatioTemporalConvNet blocks
temporal_kernel_size = 3  # Size of the temporal convolution kernel
spatial_kernel_size = 3   # Order of the spatial diffusion process
norm='layer'
gated=True

input_size = torch_dataset.n_channels
n_nodes = torch_dataset.n_nodes
horizon = torch_dataset.horizon
exog_size = torch_dataset.input_map.u.shape[-1] if 'u' in torch_dataset else 0

In [87]:
model = TimeAndGraphAnisoModel(
    input_size=input_size,
    horizon=horizon,
    n_nodes=n_nodes,
    output_size=input_size,
    exog_size=exog_size,
    hidden_size=hidden_size
)

# Print the model architecture
print(model)

TimeAndGraphAnisoModel(
  (emb): None
  (encoder): Linear(in_features=12, out_features=32, bias=True)
  (decoder): MLPDecoder(
    (readout): MLP(
      (mlp): Sequential(
        (0): Dense(
          (affinity): Linear(in_features=32, out_features=32, bias=True)
          (activation): ELU(alpha=1.0)
          (dropout): Identity()
        )
      )
      (readout): Linear(in_features=32, out_features=6, bias=True)
    )
    (rearrange): Rearrange('b n (h f) -> b h n f', f=1, h=6)
  )
  (stmp_conv): GraphAnisoGRU(cell=GraphAnisoGRUCell, return_only_last_state=True)
)


In [88]:
def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)

In [89]:
print_model_size(model)

Number of model (TimeAndGraphAnisoModel) parameters:     23657


In [90]:
from tsl.metrics.torch import MaskedMSE, MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

# Define the loss function
loss_fn = MaskedMSE()

# Setup metrics
metrics = {
    'mse': MaskedMSE(),
    'mae': MaskedMAE(),
    'mape': MaskedMAPE(),
    'mse_at_3': MaskedMSE(at=2),  # '2' indicates the third time step
    'mse_at_6': MaskedMSE(at=5)
}

# Setup predictor
predictor = Predictor(
    model=model,
    optim_class=torch.optim.Adam,
    optim_kwargs={'lr': 0.001},
    loss_fn=loss_fn,
    metrics=metrics
)


In [91]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

early_stop_callback = EarlyStopping(
    monitor='val_mse',
    patience=30,
    mode='min'
)

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mse',  # Change this to 'val_mse'
    mode='min',
)

# Setup trainer
trainer = pl.Trainer(max_epochs=100,
                    #  logger=logger,
                    #  limit_train_batches=100,  # end an epoch after 200 updates
                     callbacks=[early_stop_callback, checkpoint_callback],
                     log_every_n_steps=2,
                     gradient_clip_val=1.0,    # Prevent exploding gradients
                     precision=16
                     )


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/lightning_fabric/connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [92]:
# Set float32 matmul precision to 'medium' or 'high'
torch.set_float32_matmul_precision('medium')

In [93]:
import multiprocessing
multiprocessing.set_start_method('spawn', force=True)
trainer.fit(predictor, datamodule=datamodule)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /teamspace/studios/this_studio/logs exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name          | Type                   | Params | Mode 
-----------------------------------------------------------------
0 | loss_fn       | MaskedMSE              | 0      | train
1 | train_metrics | MetricCollection       | 0      | train
2 | val_metrics   | MetricCollection       | 0      | train
3 | test_metrics  | MetricCollection       | 0      | train
4 | model         | TimeAndGraphAnisoModel | 23.7 K | train
-----------------------------------------------------------------
23.7 K    Trainable params
0         Non-trainable params
23.7 K    Total params
0.095     Total estimated model params size (MB)
81        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
Arguments ['distances', 'metadata'] are filtered out. Only args ['u', 'edge_index', 'edge_weight', 'x'] are forwarded to the model (TimeAndGraphAnisoModel).
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f50500ddd80>
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1441, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/selectors.py", lin

In [ ]:
predictor.load_model(checkpoint_callback.best_model_path)
predictor.freeze()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/tsl/engines/predictor.py:121: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

Predictor with already instant

In [ ]:
predictor.freeze()

In [ ]:
trainer.test(predictor, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

Arguments ['metadata', 'distances'] are filtered out. Only args ['edge_weight', 'x', 'u', 'edge_index'] are forwarded to the model (TimeAndGraphAnisoModel).


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.6637526750564575     │
│         test_mae          │    0.7420239448547363     │
│         test_mape         │        14137686.0         │
│         test_mse          │    0.6637526750564575     │
│       test_mse_at_3       │    0.7337833642959595     │
│       test_mse_at_6       │    0.1788492351770401     │
└───────────────────────────┴───────────────────────────┘

[{'test_mae': 0.7420239448547363,
  'test_mape': 14137686.0,
  'test_mse': 0.6637526750564575,
  'test_mse_at_3': 0.7337833642959595,
  'test_mse_at_6': 0.1788492351770401,
  'test_loss': 0.6637526750564575}]

In [82]:
trainer.validate(predictor, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.6622081995010376     │
│          val_mae          │    0.7431228756904602     │
│         val_mape          │        14315265.0         │
│          val_mse          │    0.6622081995010376     │
│       val_mse_at_3        │    0.7360861897468567     │
│       val_mse_at_6        │    0.18198302388191223    │
└───────────────────────────┴───────────────────────────┘

[{'val_mae': 0.7431228756904602,
  'val_mape': 14315265.0,
  'val_mse': 0.6622081995010376,
  'val_mse_at_3': 0.7360861897468567,
  'val_mse_at_6': 0.18198302388191223,
  'val_loss': 0.6622081995010376}]

In [83]:
predictions_test = trainer.predict(predictor, datamodule.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [258]:
# predictions_test

In [85]:
splitter.indices

{'train': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160]),
 'val': array([173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,


In [86]:
dates = pd.to_datetime(ds['date'].values)

In [87]:
dates

DatetimeIndex(['2010-01-01', '2010-01-31', '2010-02-01', '2010-02-28',
               '2010-03-01', '2010-03-31', '2010-04-01', '2010-04-30',
               '2010-05-01', '2010-05-31',
               ...
               '2022-07-31', '2022-08-01', '2022-08-31', '2022-09-01',
               '2022-09-30', '2022-10-01', '2022-10-31', '2022-11-01',
               '2022-11-30', '2022-12-01'],
              dtype='datetime64[ns]', length=311, freq=None)

In [88]:
datamodule.splitter.indices

{'train': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160]),
 'val': array([173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,


In [89]:
import numpy as np
import pandas as pd
import torch
import xarray as xr

# 1. Load Metadata and Dates
metadata_array = np.load('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.npy')  # Shape: (21960, 2)
latitudes = metadata_array[:, 0]
longitudes = metadata_array[:, 1]
num_nodes = len(latitudes)

# Load the original dataset
# ds = xr.open_dataset('path_to_your_dataset.nc')  # Replace with your actual path
dates = pd.to_datetime(ds['date'].values)
total_time_steps = len(dates)

# Initialize splitter
# splitter = TemporalSplitter(val_len=0.35, test_len=0.1)

# Assume you have a SpatioTemporalDataset instance
# dataset = your_spatio_temporal_dataset  # Replace with your actual dataset variable

# Fit the splitter to the dataset
# splitter.fit(dataset)

# 2. Correctly Retrieve the Split Indices
split_indices = datamodule.splitter.indices
train_indices = split_indices['train']
val_indices = split_indices['val']
test_indices = split_indices['test']

print(f"Train indices: {train_indices}")
print(f"Validation indices: {val_indices}")
print(f"Test indices: {test_indices}")

# 3. Process Predictions and Actual Values
y_list = []
y_hat_list = []
mask_list = []

for batch in predictions_test:
    y_tensor = batch['y']
    y_hat_tensor = batch['y_hat']
    mask_tensor = batch['mask']
    
    y_np = y_tensor.cpu().numpy().squeeze(-1)      # Shape: (batch_size, steps, nodes)
    y_hat_np = y_hat_tensor.cpu().numpy().squeeze(-1)
    mask_np = mask_tensor.cpu().numpy().squeeze(-1)
    
    y_list.append(y_np)
    y_hat_list.append(y_hat_np)
    mask_list.append(mask_np)

# Concatenate along the batch dimension
y = np.concatenate(y_list, axis=0)         # Shape: (num_test_batches, steps, nodes)
y_hat = np.concatenate(y_hat_list, axis=0) # Shape: (num_test_batches, steps, nodes)
mask = np.concatenate(mask_list, axis=0)   # Shape: (num_test_batches, steps, nodes)

print(f"Shape of y: {y.shape}")
print(f"Shape of y_hat: {y_hat.shape}")
print(f"Shape of mask: {mask.shape}")

# 4. Map Node IDs to Geographical Coordinates
num_steps = y.shape[1]          # Number of prediction steps (e.g., 6)
num_test_dates = y.shape[0]     # Number of test dates (e.g., 25)

# Create node_ids, lat, lon repeated for each step
node_ids = np.tile(np.arange(num_nodes), num_test_dates * num_steps)  # Shape: (3,294,000,)
lat_column = np.tile(latitudes, num_test_dates * num_steps)
lon_column = np.tile(longitudes, num_test_dates * num_steps)

print(f"Shape of node_ids: {node_ids.shape}")
print(f"Shape of lat_column: {lat_column.shape}")
print(f"Shape of lon_column: {lon_column.shape}")

# 5. Align Predictions with Dates

# Repeat each test date for the number of prediction steps
test_dates_steps = np.repeat(dates[test_indices], num_steps)  # Shape: (25 * 6,) => (150,)

# Now, repeat each date for all nodes to align with the predictions
all_dates_expanded = np.repeat(test_dates_steps, num_nodes)   # Shape: (150 * 21960,) => (3,294,000,)

print(f"Shape of test_dates_steps: {test_dates_steps.shape}")       # (150,)
print(f"Shape of all_dates_expanded: {all_dates_expanded.shape}") # (3,294,000,)

# 6. Flatten the Predictions and Mask
y_flat = y.flatten()         # Shape: (3,294,000,)
y_hat_flat = y_hat.flatten()
mask_flat = mask.flatten()

print(f"Shape of y_flat: {y_flat.shape}")
print(f"Shape of y_hat_flat: {y_hat_flat.shape}")
print(f"Shape of mask_flat: {mask_flat.shape}")

# 7. Apply Mask to Filter Valid Entries
valid_indices = mask_flat.astype(bool)  # Ensure mask is boolean

filtered_dates = all_dates_expanded[valid_indices]  # Shape: (num_valid_entries,)
filtered_lat = lat_column[valid_indices]
filtered_lon = lon_column[valid_indices]
filtered_y = y_flat[valid_indices]
filtered_y_hat = y_hat_flat[valid_indices]
filtered_node_ids = node_ids[valid_indices]

print(f"Shape of filtered_dates: {filtered_dates.shape}")
print(f"Shape of filtered_lat: {filtered_lat.shape}")
print(f"Shape of filtered_lon: {filtered_lon.shape}")
print(f"Shape of filtered_y: {filtered_y.shape}")
print(f"Shape of filtered_y_hat: {filtered_y_hat.shape}")
print(f"Shape of filtered_node_ids: {filtered_node_ids.shape}")

# 8. Create the DataFrame
df_predictions = pd.DataFrame({
    'date': filtered_dates,
    'lat': filtered_lat,
    'lon': filtered_lon,
    'node_id': filtered_node_ids,
    'y': filtered_y,
    'y_hat': filtered_y_hat
})

print(df_predictions.head())
print(df_predictions.shape)


Train indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160]
Validation indices: [173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208
 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226
 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244
 245 246 247 248 24

In [90]:
df = df_predictions

In [ ]:
df 

In [76]:
import plotly.graph_objects as go
import plotly.subplots as sp
import pandas as pd
import numpy as np

def visualize_predictions_plotly(predictions, metadata, idx, horizon=None, 
                                 batches_to_visualize=None, variables_to_visualize=None):
    """
    Visualize predicted vs actual values using Plotly for specified batches and variables with variable-specific normalization.
    
    Parameters:
    - predictions (dict): Contains 'y' and 'y_hat' tensors.
    - metadata (pd.DataFrame): DataFrame with 'lat' and 'lon' columns.
    - idx (int): Index of the item being visualized.
    - horizon (int, optional): Number of time steps to visualize. Defaults to the maximum horizon in data.
    - batches_to_visualize (list or range, optional): List of batch indices to visualize. Defaults to all batches.
    - variables_to_visualize (list or range, optional): List of variable indices to visualize. Defaults to all variables.
    """
    # Extract predictions and actual values
    y_hat = predictions['y_hat'].squeeze().numpy()  # Shape: [batch, horizon, spatial_dim, variables]
    y = predictions['y'].squeeze().numpy()          # Shape: [batch, horizon, spatial_dim, variables]

    # Ensure y_hat and y have the same shape
    assert y_hat.shape == y.shape, "Predictions and actual values must have the same shape"

    # Determine the horizon if not provided
    if horizon is None:
        horizon = y_hat.shape[1]

    # Determine the number of variables
    if y_hat.ndim == 3:
        # Shape: [batch, horizon, spatial_dim]
        num_variables = 1
        y_hat = y_hat[..., np.newaxis]  # Add a variables dimension
        y = y[..., np.newaxis]
    else:
        num_variables = y_hat.shape[-1]

    if variables_to_visualize is None:
        variables_to_visualize = list(range(num_variables))
    else:
        # Ensure the variable indices are within the correct range
        variables_to_visualize = [v for v in variables_to_visualize if v < num_variables]
        if not variables_to_visualize:
            raise ValueError("No valid variables to visualize. Check 'variables_to_visualize' indices.")

    # Get latitude and longitude from metadata
    lats = metadata['lat'].values
    lons = metadata['lon'].values

    # Ensure that the number of spatial points matches the number of lats and lons
    spatial_dim = y_hat.shape[2]
    if spatial_dim != len(lats):
        raise ValueError(f"The number of spatial points in predictions ({spatial_dim}) does not match the number of locations in metadata ({len(lats)}).")

    # Calculate the min and max for lats and lons for setting map extent
    lat_min, lat_max = lats.min(), lats.max()
    lon_min, lon_max = lons.min(), lons.max()

    # If batches_to_visualize is None, visualize all batches
    if batches_to_visualize is None:
        batches_to_visualize = range(y_hat.shape[0])
    else:
        # Ensure the batch indices are within the correct range
        batches_to_visualize = [b for b in batches_to_visualize if b < y_hat.shape[0]]
        if not batches_to_visualize:
            raise ValueError("No valid batches to visualize. Check 'batches_to_visualize' indices.")

    # Precompute min and max for each variable in variables_to_visualize
    var_min_max = {}
    for var in variables_to_visualize:
        var_data_y = y[..., var]  # Shape: [batch, horizon, spatial_dim]
        var_data_y_hat = y_hat[..., var]  # Shape: [batch, horizon, spatial_dim]
        min_val = min(np.min(var_data_y), np.min(var_data_y_hat))
        max_val = max(np.max(var_data_y), np.max(var_data_y_hat))
        # Handle case where min and max are equal to avoid division by zero
        if min_val == max_val:
            min_val -= 1  # Arbitrary adjustment
            max_val += 1
        var_min_max[var] = (min_val, max_val)

    # If variables_to_visualize is [2], ensure that variable index 2 exists
    # (0-based indexing)
    for var in variables_to_visualize:
        if var >= num_variables:
            raise ValueError(f"Variable index {var} is out of range for data with {num_variables} variables.")

    # Loop through each batch and variable
    for batch in batches_to_visualize:
        for var in variables_to_visualize:
            frames = []
            slider_steps = []

            # Retrieve normalization for the current variable
            min_val, max_val = var_min_max[var]

            # Create frames for animation
            for week in range(horizon):
                # Predicted data for current week
                pred_data = go.Scattergeo(
                    lon=lons,
                    lat=lats,
                    mode='markers',
                    marker=dict(
                        size=6,
                        color=y_hat[batch, week, :, var],
                        colorscale='RdYlBu_r',
                        cmin=min_val,
                        cmax=max_val,
                        colorbar=dict(title='Predicted'),
                        opacity=0.8
                    ),
                    name='Predicted',
                    showlegend=False
                )

                # Actual data for current week
                actual_data = go.Scattergeo(
                    lon=lons,
                    lat=lats,
                    mode='markers',
                    marker=dict(
                        size=6,
                        color=y[batch, week, :, var],
                        colorscale='RdYlBu_r',
                        cmin=min_val,
                        cmax=max_val,
                        colorbar=dict(title='Actual'),
                        opacity=0.8
                    ),
                    name='Actual',
                    showlegend=False
                )

                # Define frame data
                frame = go.Frame(
                    data=[pred_data, actual_data],
                    name=f'Week {week + 1}'
                )
                frames.append(frame)

                # Create slider step for this frame
                slider_step = dict(
                    method='animate',
                    args=[[f'Week {week + 1}'],
                          dict(mode='immediate',
                               frame=dict(duration=500, redraw=True),
                               transition=dict(duration=0))],
                    label=f'Week {week + 1}'
                )
                slider_steps.append(slider_step)

            # Initial data (first frame)
            pred_data_initial = go.Scattergeo(
                lon=lons,
                lat=lats,
                mode='markers',
                marker=dict(
                    size=6,
                    color=y_hat[batch, 0, :, var],
                    colorscale='RdYlBu_r',
                    cmin=min_val,
                    cmax=max_val,
                    colorbar=dict(title='Predicted'),
                    opacity=0.8
                ),
                name='Predicted',
                showlegend=True
            )

            actual_data_initial = go.Scattergeo(
                lon=lons,
                lat=lats,
                mode='markers',
                marker=dict(
                    size=6,
                    color=y[batch, 0, :, var],
                    colorscale='RdYlBu_r',
                    cmin=min_val,
                    cmax=max_val,
                    colorbar=dict(title='Actual'),
                    opacity=0.8
                ),
                name='Actual',
                showlegend=True
            )

            # Create subplots with two geo subplots side by side
            fig = sp.make_subplots(
                rows=1, cols=2,
                specs=[[{'type': 'scattergeo'}, {'type': 'scattergeo'}]],
                subplot_titles=("Predicted", "Actual")
            )

            # Add initial traces to respective subplots
            fig.add_trace(pred_data_initial, row=1, col=1)
            fig.add_trace(actual_data_initial, row=1, col=2)

            # Add frames
            fig.frames = frames

            # Define sliders
            fig.update_layout(
                title_text=f'Item {idx + 1}, Batch {batch + 1}, Variable {var + 1} - Predicted vs Actual EDDI',
                updatemenus=[dict(
                    type='buttons',
                    showactive=False,
                    buttons=[dict(label='Play',
                                  method='animate',
                                  args=[None, {"frame": {"duration": 500, "redraw": True},
                                               "fromcurrent": True,
                                               "transition": {"duration": 0}}],
                                  )],
                    x=0.1,
                    y=1.15
                )],
                sliders=[dict(
                    active=0,
                    pad={"t": 50},
                    steps=slider_steps,
                    currentvalue=dict(
                        visible=True,
                        prefix="Week: ",
                        xanchor="right",
                        font=dict(size=14, color="#666")
                    ),
                )],
                geo=dict(
                    scope='world',
                    projection_type='natural earth',
                    showland=True,
                    landcolor='lightgray',
                    showcountries=True,
                    countrycolor='black',
                    lataxis=dict(range=[lat_min - 1, lat_max + 1]),
                    lonaxis=dict(range=[lon_min - 1, lon_max + 1]),
                ),
                geo2=dict(
                    scope='world',
                    projection_type='natural earth',
                    showland=True,
                    landcolor='lightgray',
                    showcountries=True,
                    countrycolor='black',
                    lataxis=dict(range=[lat_min - 1, lat_max + 1]),
                    lonaxis=dict(range=[lon_min - 1, lon_max + 1]),
                )
            )

            # Update frames to assign traces to correct subplots
            for frame in fig.frames:
                week_idx = int(frame.name.split()[1]) - 1
                frame.data = [
                    go.Scattergeo(
                        lon=lons,
                        lat=lats,
                        mode='markers',
                        marker=dict(
                            size=6,
                            color=y_hat[batch, week_idx, :, var],
                            colorscale='RdYlBu_r',
                            cmin=min_val,
                            cmax=max_val,
                            opacity=0.8
                        ),
                        showlegend=False
                    ),
                    go.Scattergeo(
                        lon=lons,
                        lat=lats,
                        mode='markers',
                        marker=dict(
                            size=6,
                            color=y[batch, week_idx, :, var],
                            colorscale='RdYlBu_r',
                            cmin=min_val,
                            cmax=max_val,
                            opacity=0.8
                        ),
                        showlegend=False
                    )
                ]

            # Update layout for better appearance
            fig.update_layout(
                height=600,
                width=1200,
                margin=dict(l=50, r=50, t=100, b=50)
            )

            # Display the figure
            fig.show()

            # Optionally, save the figure to an HTML file
            # fig.write_html(f'item_{idx+1}_batch_{batch+1}_variable_{var+1}.html')

# Example Usage

# # Assuming you have the metadata DataFrame available
# metadata = pd.read_parquet('ml-drought-forecasting/ml-modeling-pipeline/data/05_model_input/metadata.parquet')

# # Ensure that predictions_test is a list of dictionaries with 'y' and 'y_hat'
# # For example:
# # predictions_test = [{'y': tensor1, 'y_hat': tensor2}, {'y': tensor3, 'y_hat': tensor4}, ...]

# # Specify the indices of the items you want to visualize
# items_to_visualize = [1]  # Replace with desired item indices (0-based indexing)
# batches_to_visualize = [0, 1]  # Replace with desired batch indices within each item
# variables_to_visualize = [0]  # Replace with desired variable indices (e.g., if multiple EDDI metrics)

# # Call the function for specified items in predictions_test
# for idx in items_to_visualize:
#     pred = predictions_test[idx]
#     visualize_predictions_plotly(
#         pred, metadata, idx, horizon=pred['y'].shape[1],
#         batches_to_visualize=batches_to_visualize, 
#         variables_to_visualize=variables_to_visualize
#     )
#     print(f"Processed item {idx + 1}/{len(predictions_test)}")


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.stats import genlogistic, norm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Assuming your dataset is loaded into ds

# Convert units from meters to millimeters
ds['tp_mm'] = ds['tp'] * 1000  # Convert to mm
ds['pev_mm'] = ds['pev'] * 1000  # Convert to mm

# Calculate water balance D = P - PET
ds['D'] = ds['tp_mm'] - ds['pev_mm']

# Set the time scale for SPEI calculation
scale = 12  # 12 months

# Aggregate over the time scale
D_rolling = ds['D'].rolling(date=scale, center=False).sum() #.dropna('date')

# Define the function to calculate SPEI
def calculate_spei(series):
    # Remove NaN values
    series = series[~np.isnan(series)]
    
    if len(series) < 3:
        # Not enough data to fit the distribution
        return np.full(series.shape, np.nan)
    
    # Fit the log-logistic distribution
    try:
        c, loc, scale_param = genlogistic.fit(series)
    except Exception as e:
        # If fitting fails, return NaNs
        return np.full(series.shape, np.nan)
    
    # Calculate cumulative probabilities
    probs = genlogistic.cdf(series, c, loc=loc, scale=scale_param)
    
    # Ensure probabilities are within valid range
    probs = np.clip(probs, 1e-6, 1 - 1e-6)
    
    # Convert cumulative probabilities to SPEI values (standard normal variates)
    spei_values = norm.ppf(probs)
    
    return spei_values

# Ensure the data is chunked appropriately for dask parallelization
D_rolling_chunked = D_rolling.chunk({'latitude': 10, 'longitude': 10})

# Apply the function across all grid points
spei = xr.apply_ufunc(
    calculate_spei,
    D_rolling_chunked,
    input_core_dims=[['date']],
    output_core_dims=[['date']],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
    keep_attrs=True
)

# Compute the results
spei = spei.compute()

# Assign coordinates and attributes
spei = spei.assign_coords({
    'latitude': ds.latitude,
    'longitude': ds.longitude,
    'date': D_rolling.date
})

spei.name = f'SPEI_{scale}'
spei.attrs['long_name'] = f'Standardized Precipitation Evapotranspiration Index ({scale}-month scale)'
spei.attrs['units'] = 'Standard Deviations'

# Visualize the SPEI for a specific date
plot_date = spei.date.values[-1]

plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
spei.sel(date=plot_date).plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap='RdBu',
    vmin=-3,
    vmax=3,
    cbar_kwargs={'label': 'SPEI'}
)
ax.coastlines()
ax.set_title(f'SPEI ({scale}-month) on {pd.to_datetime(str(plot_date)).strftime("%Y-%m")}')
plt.show()

In [ ]:
ds = xr.merge([ds, spei])
ds = ds.drop_vars(["tp_mm", "pev_mm", "D"])